# User input

## Date & Zone info

In [6]:
# Define date of the previous of the day ahead prediction day (i.e. starting_date = '04/01/2024' or comment it out for auto getting today's date)
# starting_date = '03/02/2024'

# Define how many day(s) forward to predict (i.e. days_ahead = 1 means one day (24hours) ahead forecast)
days_ahead = 1

# Define the list with the bidding zone(s) for which to create predictions (i.e. zones = ['DE','FR','HU'])
zones = ['DE']

## Data info

In [7]:
# state whether to include the Naive model as forecasting model
Naive_model = False
# state whether to run and include the Time series models
Time_series = True
# Define start date for Timeseries
startDate_TS = '2022-06-09'
# define the number of days to look back from today for the evaluation step (Best_Hourly model)
daysBack = 20

# Define the training version of the models
Training_version = 'v1'
# Define the tables and their columns in MySQL that match those in the model training script
Countries_dict = {
  	'v1':{
  	  	'DE' : {
			'Fundamentals_forecast': ['Load_DE', 'Wind_DE', 'Solar_DE','Hydro_DE','Nuc_DE'],
			'GasSpot': ['THE'],
			'EUA': ['EUA_DecLastPrice'],
			'Prices': ['DE_LU'] },
}}
# Define tables with daily granularity in order to convert them to hourly granularity
daily_granularity = ['GasSpot', 'EUA','GasFutures']
# Define data Table from which the single prediction column is predicted
TablePred = 'Prices'

## Script info

In [8]:
# state whether to create the evaluation plots
create_plots = False
# state whether to notify via Teams over the forecasts completion for the current and for all zones
Teams_notification = True

# Libraries, connections & functions

In [9]:
#------- Libraries
# Time
from datetime import datetime, date, timedelta
import holidays
# Data
import pandas as pd
import math
import scipy
import numpy as np
from numpy import arange, mean, std
from functools import reduce
import collections
import json
from keras.models import model_from_json
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
# graphics
import seaborn as sns
import matplotlib.pyplot as plt
from jupyterthemes import jtplot
import plotly.express as px
import plotly.graph_objects as go
import pandasgui
import pygwalker as pyg
# ML
from tensorflow import keras
# Timeseries
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.holtwinters import ExponentialSmoothing
import pmdarima as pm
from scalecast.Forecaster import Forecaster
from tbats import TBATS
from ThymeBoost import ThymeBoost as tb
# System
import os
import pickle
import requests
import PySimpleGUI as sg
import warnings
# Turning off the Deprecation warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.simplefilter(action='ignore', category=UserWarning)
warnings.simplefilter(action='ignore', category=FutureWarning)

# helper script for the evaluation of Best_Hourly model
import Models_Evaluation

# helper script containing several functions to communicate with MySQL
import MySQL_functions
# connection info to connect to MySQL server
MySQL_info = { 'user':'python',
				'passw':'sentrade_admin',
				'host':'172.24.70.213',
				'port' : 3306,
				'database_name':'MORE_Trading' }

# helper script to define users folder pathes
import folders
Path_dict = folders.folder()
ModelPathFolder = Path_dict['Models']

#------- Functions
# define function to make predictions
def ModelsForecast(model_name, X_forecasts):
	# model names list
	RNN_model_names = ['LSTM','BILSTM','GRU','LSTM1','LSTM2','LSTM3','LSTM4','GRU1','GRU2','GRU3','BILSTM1','BILSTM2','BILSTM3']	
	# load model ( different loading based if it is a NN or not)
	if model_name in RNN_model_names:
		# Model file location
		FilePath_JSON = ModelPathFolder + project + model_name + '.JSON'
		FilePath_h5 = ModelPathFolder + project + model_name + '.h5'
		# load json and create model
		json_file = open(FilePath_JSON, 'r')
		loaded_model_json = json_file.read()
		json_file.close()
		model = model_from_json(loaded_model_json)
		# load weights into new model
		model.load_weights(FilePath_h5)
	else:		
		# file location on disk
		ModelFilePath = ModelPathFolder + project + model_name + '.sav'
		# load the model
		model = pickle.load(open(ModelFilePath, 'rb'))
		
	# load scaler for X Forecast features
	ScalerFilePath = ModelPathFolder + project + 'X_scaler.sav'
	scaler = pickle.load(open(ScalerFilePath , 'rb'))
	# extraxt all the column names with order in which they were scaled and reorder the pandas dataframe
	X_forecasts.columns = scaler.get_feature_names_out()
	# scale Forecast features
	X_forecasts_scaled = scaler.transform(X_forecasts)
	
	# make 3D matrix for RNNs
	if model_name in RNN_model_names:
		time_steps=1
		Xs = []
		for i in range(len(X_forecasts_scaled) - time_steps):
			v = X_forecasts_scaled[i:i + time_steps, :]
			Xs.append(v)
		X_forecasts_scaled_final = np.array(Xs)
	else:
		X_forecasts_scaled_final = X_forecasts_scaled

	# Make prediction
	Y_forecasts_model = model.predict(X_forecasts_scaled_final)
		
	# ------ Scale Y and other transformation on the predictions ------ 
	if model_name in RNN_model_names:
		# load the Y scaler from disk
		ScalerFilePath = ModelPathFolder + project + 'Y_scaler.sav'
		scaler_Y = pickle.load(open(ScalerFilePath, 'rb'))
		Y_forecasts = scaler_Y.inverse_transform(Y_forecasts_model.reshape(-1, 1))
		# as the last prediction value is non defined mask it with the latest one
		Y_forecasts = np.insert(Y_forecasts,len(Y_forecasts),Y_forecasts[-1])
	elif model_name == 'MLP':
		Y_forecasts =  pd.Series( Y_forecasts_model.flatten() )
	else:
		Y_forecasts =  Y_forecasts_model
		
	# return predictions
	return Y_forecasts.astype(float).round(2)


# function to post the notification to Teams
def send_teams(webhook_url:str, content:str, title:str, color:str="#bfbf9f"):
	response_TEAMS = requests.post(
		url=webhook_url,
		headers={"Content-Type": "application/json"},
		json={
			"themeColor": color,
			"summary": title,
			"sections": [{
				"activityTitle": title,
				"activitySubtitle": content
			}],
		},
	)
	if response_TEAMS.status_code == 200:
		posted = 'Posted to Teams'
	else:
		posted = 'Error - not posted to Teams'
	return posted

#------- 
# initiate dictionary to store suggested models and predictions for all zones and days ahead
SuggPredictions = collections.defaultdict(lambda : collections.defaultdict(dict))

# ML Model names list to run
model_names = ['LinearRegression','Lasso','Ridge','kNN','XGB','RandomForest','GAM','SVM','GBM','AdaBoost','CatBoost','EXT','MLP','LSTM','BILSTM','GRU','LSTM1','LSTM2','LSTM3','LSTM4','GRU1','GRU2','GRU3','BILSTM1','BILSTM2','BILSTM3']


# Run predictions

In [ ]:
#------- Zones loop
for zone in zones:

	# get country's holidays to decide whether to include naive model
	holidaysDates = holidays.country_holidays(zone)

	# define evaluation folder
	EvaluationPathFolder = Path_dict['Evaluation']+zone+'\\'+TablePred+'\\'+Training_version+'\\'

	# define project name
	project = zone + '_' + TablePred + '_' + Training_version + '_'
	print('Current project name:', project)

	# define Price name column
	ColumPred = Countries_dict[Training_version][zone]['Prices'][0]

	#------- Dates ahead loop
	for day_loop in range(0, days_ahead):

	#------- Dates
		# Define start and end date to retrieve data from MySQL database
		try:
			start_time = pd.to_datetime(starting_date, format='%d/%m/%Y') + timedelta(days=day_loop)
		except NameError as ne:
			print('Using current datetime..')
			start_time = datetime.now() + timedelta(days=day_loop)

		# define today's date
		Today = pd.to_datetime(start_time.strftime('%Y-%m-%d'), format='%Y-%m-%d')

	   		
		# define day ahead date start timestamp
		start = (Today + timedelta(days=1) ).strftime('%Y-%m-%d %H:%M:%S')
		# define day ahead date end
		end = (Today + timedelta(days=1) + timedelta(hours=23)).strftime('%Y-%m-%d %H:%M:%S')
		print('\nPrediction starting timestamp:',start,' - ending timestamp:',end)

		# Initiate data frame to save forecasts
		Forecasts = pd.DataFrame(pd.date_range(start, periods=24, freq='H'), columns=['Datetime'])
		Forecasts['project'] = project
		
		# define table name for forecasted data
		TableForecasts = TablePred + '_forecast'


	#------- Import data
		# concatenate column and table names
		Tables = []
		Columns = []
		for table, columns in Countries_dict[Training_version][zone].items():
			Columns += columns
			Tables.append(table)
		
		# initiate list to store dataframe created on the loop
		list_of_dfs = []
		
		# loop through countries dictionary and retrieve data from MySQL
		for table, columns in Countries_dict[Training_version][zone].items():
			# Table 'GasSpot' has a peculiar structure and needs different querry format
			if table == 'GasSpot':
				query = f'SELECT DeliveryStart, SettlementPrice as {columns[0]} FROM MORE_Trading.GasSpot where Market="{columns[0]}" AND DeliveryStart <= "{start}" ORDER BY DeliveryStart DESC LIMIT 1;'
				DF = MySQL_functions.ExecuteQuery(server_credentials = MySQL_info, query=query)
				DF.rename(columns={'DeliveryStart':'Datetime'}, inplace=True, errors='ignore')
			elif table == "EUA":
				query = f'SELECT Datetime, EUA_DecLastPrice FROM MORE_Trading.EUA WHERE EUA_DecLastPrice IS NOT NULL AND Datetime <= "{start}" ORDER BY Datetime DESC LIMIT 1;'
				DF = MySQL_functions.ExecuteQuery(server_credentials = MySQL_info, query=query)
				DF.rename(columns={'DeliveryStart':'Datetime'}, inplace=True, errors='ignore')
			elif table == 'Prices':
				Prices = MySQL_functions.SelectData(server_credentials = MySQL_info, table_name=TablePred, columns=ColumPred, start=startDate_TS, end=Today.strftime('%Y-%m-%d %H:%M:%S'))
				# set all data to float
				Prices[ColumPred] = Prices[ColumPred].apply(pd.to_numeric, errors='coerce', downcast='float')
				# sort values by datetime
				Prices["Datetime"] = pd.to_datetime(Prices["Datetime"], format='%Y-%m-%d %H:%M:%S')
				Prices.sort_values(by="Datetime", inplace=True)
				# fill na values (cases of time change)
				Prices = Prices.fillna(method='ffill')
			else:
				DF = MySQL_functions.SelectData(server_credentials = MySQL_info, table_name=table, columns=columns, start=start,end=end)
			
			# print(table, columns,'\n',DF)
			# check and then convert daily to hourly timestamp
			if table in daily_granularity:
				for column in columns:
					# create dataframe with day ahead datetime
					df = pd.DataFrame(index=pd.date_range(start, end, freq="H"))
					df.index.name = 'Datetime'
					df[column] = DF[column][0]
					# add dataframe to the respective list
					list_of_dfs.append(df)
			elif table == 'Prices':
				pass
			else:
				# set Datetime as datetime object
				DF['Datetime'] = pd.to_datetime(DF['Datetime'], format='%Y-%m-%d %H:%M:%S')
				# add dataframe to the respective list
				list_of_dfs.append(DF)
		
		# Merge dataframes from the list to a single dataframe
		Data = reduce(lambda x, y: pd.merge(x,y, on='Datetime'), list_of_dfs)
				
		# Show all missing values
		print('\nMissing values:\n', Data.isnull().sum(),
			'\nTable size:', Data.shape)
		
		# Replace NULL values with the value from the ffill: previous row bfill: next row
		data = Data.fillna(method='ffill').fillna(method='bfill')
		print('\nAll NaN values were imputated')
		# set Datetime as index
		data.set_index('Datetime', inplace=True)
		# set all data to float
		data = data.apply(pd.to_numeric, errors='coerce', downcast='float')
		# ------- 
		# show data statisticss
		# print(data.describe())
		# append data to excel to keep historical forecasts
		data.to_csv(EvaluationPathFolder+'Attributes forecasts.csv', mode='a',index=True, header=False)
		# show data
		print(data)

	#------- ML models forecasts	
		# Run model's list
		for model in model_names:
			print('Model running:', model)
			Forecasts[model] = ModelsForecast(model, data)


	#------- Naive and Timeseries models checks for inclusion

		# 1.Check dates for holidays and weekends in order to decide whether Naive prices is suitable
		# check for holiday for Today
		if (Today.strftime('%A') in [ 'Saturday','Sunday']) | (Today.strftime('%Y-%m-%d') in holidaysDates):
			Today_holiday = True
		else:
			Today_holiday = False
		# check for holiday for the DayAhead
		if ((Today + timedelta(days=1)).strftime('%A') in [ 'Saturday','Sunday']) | ((Today + timedelta(days=1)).strftime('%Y-%m-%d') in holidaysDates):
			DayAhead_holiday = True
		else:
			DayAhead_holiday = False
		# if Today and DayAhead match exclude Naive model
		if Today_holiday != DayAhead_holiday:
			Naive_model = False
		else:
			pass

		# 2.Check if Prices are up to the latest day to decide for Naive and Timeseries models
		if Naive_model or Time_series:
			if list(Prices['Datetime'].iloc[-24:,] + timedelta(days=1)) != list(Forecasts['Datetime']):
				if days_ahead > 1 and day_loop != 0:
					print('Script will continue to run without Naive and Timeseries models.')
					Naive_model = False
					Time_series = False
				else:
					sg.theme('DarkAmber')   # Add a touch of color
					# All the stuff inside your window.
					layout = [[sg.Text('Hstorical prices timestamp are not up to date. If you want to continue without the Time series and Naive model press "Continue without" else "Cancel" and run respective script to fill the database and rerun from here and below.')], [sg.Button('Continue without'), sg.Button('Cancel')]]
					# Create the Window
					window = sg.Window('Price forecasting alert', layout)
					# Event Loop to process "events" and get the "values" of the inputs
					event, values = window.read()
					if event == sg.WIN_CLOSED or event == 'Cancel':
						print('Script has stopped running.')
						window.close()
						raise SystemExit(0)
					else:
						print('Script will continue to run without Naive and Timeseries models.')
						Naive_model = False
						Time_series = False
						window.close()
						pass
			else:
				pass
		else:
			pass


				
	#------- Time series models forecasts ---
		if Time_series:		
			model_name = 'SARIMA_TS'	
			# create model
			SARIMAmodel = SARIMAX(endog = Prices[ColumPred],order = (3,0,3), seasonal_order=(2,1,0,7),
								enforce_stationarity=False, enforce_invertibility=False)
			# fit model
			results = SARIMAmodel.fit()
			# Generate predictions
			Predictions_LowerUpper = results.get_forecast(24).conf_int(alpha = 0.05) 
			# store predictions in dataframe
			Forecasts[model_name] = Predictions_LowerUpper.mean(axis=1).reset_index(drop=True)
		
			model_name = 'ETS_TS'	
			# create and fit model
			ets_model = ExponentialSmoothing(endog=Prices[ColumPred], trend='additive', seasonal='additive',seasonal_periods=12,damped=True)
			# fit model
			ets_model = ets_model.fit(optimized=True, remove_bias=False)	
			# Generate predictions  
			Predictions = ets_model.forecast(24)	
			# Generate predictions
			Forecasts[model_name] = Predictions.reset_index(drop=True)

			model_name = "GBLA_TS"	
			# create and fit model
			boosted_model = tb.ThymeBoost(verbose=1)
			# fit model
			output = boosted_model.fit(Prices[ColumPred], trend_estimator=['linear', 'arima'], arima_order='auto', global_cost='mse')
			# Generate predictions  
			Forecasts[model_name] = boosted_model.predict(output, 24).reset_index(drop=True)['predictions']

			model_name = "LSTM_TS"	
			# define data
			f = Forecaster(y=Prices[ColumPred], current_dates=Prices['Datetime'])
			# observations to test the results
			f.set_test_length(24)
			# future points to forecast
			f.generate_future_dates(24)
			# create LSTM neural network
			f.set_estimator('lstm')
			# make predictions
			f.manual_forecast(call_me='lstm_best',
							lags=24,
							batch_size=32,
							epochs=20,
							validation_split=.2,
							shuffle=True,
							activation='tanh',
							optimizer='Adam',
							learning_rate=0.001,
							lstm_layer_sizes=(72,)*4,
							dropout=(0,)*4,
							plot_loss=False)	
			# Generate predictions 
			Forecasts[model_name] = f.forecast

			model_name = "Ridge_TS"
			# define data
			df = Prices
			df = df.set_index('Datetime').asfreq('h')
			# fit data to forecaster
			f = Forecaster(y=df[ColumPred], current_dates = df.index )
			# future points to forecast
			f.generate_future_dates(24)
			# add AR terms
			f.add_ar_terms(1)
			# add seasonal AR terms
			f.add_AR_terms((3,3))
			# # add seasonal regressors
			f.add_seasonal_regressors('day',raw=False,sincos=True)
			# add time regressor
			f.add_time_trend()	
			# create model
			f.set_estimator('ridge')
			# make predictions
			f.manual_forecast(alpha=0.11)	
			# Generate predictions 
			Forecasts[model_name] = f.forecast

			model_name = "Lasso_TS"	
			# define data
			df = Prices
			df = df.set_index('Datetime').asfreq('h')
			# fit data to forecaster
			f = Forecaster(y=df[ColumPred], current_dates = df.index)
			# future points to forecast
			f.generate_future_dates(24)
			# add AR terms
			f.add_ar_terms(1)
			# add seasonal AR terms
			f.add_AR_terms((3,3))
			# # add seasonal regressors
			f.add_seasonal_regressors('day',raw=False,sincos=True)
			# add time regressor
			f.add_time_trend()	
			# create model
			f.set_estimator('lasso')
			# make predictions
			f.manual_forecast(alpha=0.01)	
			# Generate predictions 
			Forecasts[model_name] = f.forecast

			model_name = "GBT_TS"	
			# define data
			df = Prices
			df = df.set_index('Datetime').asfreq('h')
			# fit data to forecaster
			f = Forecaster(y=df[ColumPred], current_dates = df.index)
			# future points to forecast
			f.generate_future_dates(24)
			# add AR terms
			f.add_ar_terms(1)
			# add seasonal AR terms
			f.add_AR_terms((3,3))
			# # add seasonal regressors
			f.add_seasonal_regressors('day',raw=False,sincos=True)
			# add time regressor
			f.add_time_trend()	
			# create model
			f.set_estimator('gbt')
			# make predictions
			f.manual_forecast(max_depth = 3, max_features = None)   
			# Generate predictions 
			Forecasts[model_name] = f.forecast

			model_name = "kNN_TS"	
			# define data
			df = Prices
			df = df.set_index('Datetime').asfreq('h')
			# fit data to forecaster
			f = Forecaster(y=df[ColumPred], current_dates = df.index)
			# future points to forecast
			f.generate_future_dates(24)
			# add AR terms
			f.add_ar_terms(1)
			# add seasonal AR terms
			f.add_AR_terms((3,3))
			# # add seasonal regressors
			f.add_seasonal_regressors('day',raw=False,sincos=True)
			# add time regressor
			f.add_time_trend()	
			# create model
			f.set_estimator('knn')
			# make predictions
			f.manual_forecast(n_neighbors = 61)	
			# Generate predictions 
			Forecasts[model_name] = f.forecast

			model_name = "MLP_TS"	
			# define data
			df = Prices
			df = df.set_index('Datetime').asfreq('h')
			# fit data to forecaster
			f = Forecaster(y=df[ColumPred], current_dates = df.index)
			# future points to forecast
			f.generate_future_dates(24)
			# add AR terms
			f.add_ar_terms(1)
			# add seasonal AR terms
			f.add_AR_terms((3,3))
			# # add seasonal regressors
			f.add_seasonal_regressors('day',raw=False,sincos=True)
			# add time regressor
			f.add_time_trend()	
			# create model
			f.set_estimator('mlp')
			# make predictions
			f.manual_forecast(activation = 'relu', hidden_layer_sizes = (25,), solver= 'lbfgs')	
			# Generate predictions 
			Forecasts[model_name] = f.forecast  

			model_name = "MLR_TS"	
			# define data
			df = Prices
			df = df.set_index('Datetime').asfreq('h')
			# fit data to forecaster
			f = Forecaster(y=df[ColumPred], current_dates = df.index)
			# future points to forecast
			f.generate_future_dates(24)
			# add AR terms
			f.add_ar_terms(1)
			# add seasonal AR terms
			f.add_AR_terms((3,3))
			# # add seasonal regressors
			f.add_seasonal_regressors('day',raw=False,sincos=True)
			# add time regressor
			f.add_time_trend()	
			# create model
			f.set_estimator('mlr')
			# make predictions
			f.manual_forecast()	
			# Generate predictions 
			Forecasts[model_name] = f.forecast

			model_name = "RF_TS"	
			# define data
			df = Prices
			df = df.set_index('Datetime').asfreq('h')
			# fit data to forecaster
			f = Forecaster(y=df[ColumPred], current_dates = df.index)
			# future points to forecast
			f.generate_future_dates(24)
			# add AR terms
			f.add_ar_terms(1)
			# add seasonal AR terms
			f.add_AR_terms((3,3))
			# # add seasonal regressors
			f.add_seasonal_regressors('day',raw=False,sincos=True)
			# add time regressor
			f.add_time_trend()	
			# create model
			f.set_estimator('rf')
			# make predictions
			f.manual_forecast(max_depth = 5, n_estimators = 100, max_features = 'sqrt', max_samples = 0.75)	
			# Generate predictions 
			Forecasts[model_name] = f.forecast

			model_name = "SVR_TS"	
			# define data
			df = Prices
			df = df.set_index('Datetime').asfreq('h')
			# fit data to forecaster
			f = Forecaster(y=df[ColumPred], current_dates = df.index)
			# future points to forecast
			f.generate_future_dates(24)
			# add AR terms
			f.add_ar_terms(1)
			# add seasonal AR terms
			f.add_AR_terms((3,3))
			# # add seasonal regressors
			f.add_seasonal_regressors('day',raw=False,sincos=True)
			# add time regressor
			f.add_time_trend()	
			# create model
			f.set_estimator('svr')
			# make predictions
			f.manual_forecast(kernel = 'linear', C = 3, epsilon = 0.01)	
			# Generate predictions 
			Forecasts[model_name] = f.forecast


			model_name = "XBD_TS"	
			# define data
			df = Prices
			df = df.set_index('Datetime').asfreq('h')
			# fit data to forecaster
			f = Forecaster(y=df[ColumPred], current_dates = df.index)
			# future points to forecast
			f.generate_future_dates(24)
			# add AR terms
			f.add_ar_terms(1)
			# add seasonal AR terms
			f.add_AR_terms((3,3))
			# # add seasonal regressors
			f.add_seasonal_regressors('day',raw=False,sincos=True)
			# add time regressor
			f.add_time_trend()	
			# create model
			f.set_estimator('xgboost')
			# make predictions
			f.manual_forecast(n_estimators = 150, scale_pos_weight = 5, learning_rate = 0.1, gamma = 5, subsample = 0.8)	
			# Generate predictions 
			Forecasts[model_name] = f.forecast

			model_name = "LXBD_TS"	
			# define data
			df = Prices
			df = df.set_index('Datetime').asfreq('h')
			# fit data to forecaster
			f = Forecaster(y=df[ColumPred], current_dates = df.index)
			# future points to forecast
			f.generate_future_dates(24)
			# add AR terms
			f.add_ar_terms(1)
			# add seasonal AR terms
			f.add_AR_terms((3,3))
			# # add seasonal regressors
			f.add_seasonal_regressors('day',raw=False,sincos=True)
			# add time regressor
			f.add_time_trend()	
			# create model
			f.set_estimator('lightgbm')
			# make predictions
			f.manual_forecast(n_estimators = 150, boosting_type = 'goss', max_depth = 2, learning_rate = 0.1)	
			# Generate predictions 
			Forecasts[model_name] = f.forecast
		else:
			pass

		if Naive_model:
			Forecasts['Naive'] = Prices[ColumPred].iloc[-24:,].values

	
	#------- Custom models
		try:
			#--- Best previous 3 days models
			# define dictionary to store the best models found from each past day. day=3 means yesterday
			Best_models = {}
			# define how many actual days back to search for best model
			days = 3
			# define how many days back to search for days having data after which to stop
			search_ending = 60
			# set iterator for searching back
			i = 0
			
			# loop until number of days are found or search iterations is completed
			while days > 0:
				# define timestamp os start and end of each past day
				day_from  = pd.to_datetime(Prices.iloc[-1,0].strftime('%Y-%m-%d'), format='%Y-%m-%d') - timedelta(days=i)
				day_from_str = (day_from).strftime('%Y-%m-%d %H:%M:%S')
				day_until = (day_from + timedelta(hours=23))
				day_until_str = (day_until).strftime('%Y-%m-%d %H:%M:%S')
				# retrieve Historical forecasts data from MySQL
				Historical_forecasts = MySQL_functions.SelectData(server_credentials = MySQL_info, table_name=TableForecasts, columns='All', start=day_from_str,end=day_until_str)
				# Keep forecasts of the same project
				Historical_forecasts = Historical_forecasts[Historical_forecasts['project'] == project]
				# if no data found iterate to the next day
				if Historical_forecasts.empty:
					print('No data found in MySQL for:',day_from_str, day_until_str)
					# exit while loop if trying for more than 10 days back
					if i == search_ending:
						break
				else:
					# initiate dictionary to store metrics
					Metrics = []
					# exclude Best models and not other not necessary columns
					columns_toDrop = ['Datetime','project','Best_Average','Best_Hourly','Best_Daily','Best_811','Best_721','Best_631','Best_532']
					ForecPrices = Historical_forecasts.drop(columns=columns_toDrop, errors='ignore')
					# Drop all columns with NaN values
					ForecPrices = ForecPrices.dropna(axis=1)
					# define data range of Actual prices which to compare with on metrics
					ActualPrices = Prices[(Prices['Datetime']>=np.datetime64(day_from)) & (Prices['Datetime']<=np.datetime64(day_until))]
					# define columns as lists					
					list1 = Forecasts.loc[:, ~Forecasts.columns.isin(['Datetime','project'])].columns
					list2 = ForecPrices.columns
		
					if set(list1).issubset(list2):
						# loop through the columns
						for column in list1:
							if ForecPrices[column].values.any() != None:
								mae = mean_absolute_error(ActualPrices[ColumPred].values, ForecPrices[column].values).astype(float).round()
								rmse = mean_squared_error(ActualPrices[ColumPred].values, ForecPrices[column].values, squared=False).astype(float).round()
								mape = mean_absolute_percentage_error(ActualPrices[ColumPred].values, ForecPrices[column].values)		  
								# store metrics to dictionary
								Metrics.append({'model':column, 'mae':mae, 'rmse': rmse, 'mape': round(mape,2)})
							else:
								pass
						# sort metrics according to mae
						sorted_list = sorted(Metrics, key=lambda d: d['mae']) 
						# add best model to relative dictionary
						for r, rank_model in enumerate(sorted_list):
							print('\n For day',days,'the best model found is', rank_model,'\n\n')
							# check if the best column found is included in today's forecasts
							if rank_model['model'] in Forecasts.columns:
								Best_models[str(days)] = rank_model['model']
								days -= 1
								break
							else:
								continue
				# increase day back search iterator
				i += 1
			
			# Save models to dataframe
			try:
				Forecasts['Best_Daily'] = Forecasts[Best_models['3']]
				Forecasts['Best_811'] = Forecasts[Best_models['3']]*0.8 + Forecasts[Best_models['2']]*0.1 + Forecasts[Best_models['1']]*0.1
				Forecasts['Best_721'] = Forecasts[Best_models['3']]*0.7 + Forecasts[Best_models['2']]*0.2 + Forecasts[Best_models['1']]*0.1
				Forecasts['Best_631'] = Forecasts[Best_models['3']]*0.6 + Forecasts[Best_models['2']]*0.3 + Forecasts[Best_models['1']]*0.1
				Forecasts['Best_532'] = Forecasts[Best_models['3']]*0.5 + Forecasts[Best_models['2']]*0.3 + Forecasts[Best_models['1']]*0.2
			except Exception as e:
				print('\nException occurred in forming Best models!\n', e)
				
				
			#--- Best Hourly
			# function to make the evaluation
			Evaluation_df = Models_Evaluation.Evaluation_project(server_credentials=MySQL_info,zone=zone, TablePred=TablePred, ColumPred=ColumPred, Training_version=Training_version,  startdate=Today, daysBack=daysBack, create_plots=create_plots)
			
			# Load Best hourly table dictionary from disc
			RankingDictionaryPathFile = ModelPathFolder + 'Best_Hourly_Table.pkl'
			with open(RankingDictionaryPathFile, 'rb') as f:
				RankingTable = pickle.load(f)
			try:
				RankingTable = RankingTable[zone][TablePred][Training_version]
				BestModelResult = []
				for i in range(0, 24, 1):
					# try successively the model based on the ranking
					for c in range(1, len(RankingTable.columns) ):
						if RankingTable.iloc[i,c] in Forecasts.columns:
							try:
								BestModelColumnName = RankingTable.iloc[i,c]
								Bestprediction = Forecasts.loc[i, BestModelColumnName]
								break
							except:
								continue
					BestModelResult.append(Bestprediction) 
				Model_result = pd.DataFrame(BestModelResult, columns=['Best_Hourly'])
				# reset index
				Model_result.reset_index(drop=True, inplace=True)
			
				# save models to dataframe
				Forecasts['Best_Hourly'] = Model_result
			except Exception as e:
					print('\nBest_Hourly model was not created due to exception:\n',e)
					pass
			
			
			#--- Average of Best models
			try:
				Forecasts['Best_Average'] = Forecasts[['Best_Hourly','Best_Daily','Best_811','Best_721','Best_631','Best_532']].mean(axis=1)
			except Exception as e:
				print('\nBest_Average model was not created due to exception:\n',e)	   
			
			
			#--- Suggested model
			for j in range(len(Evaluation_df)):
				if Evaluation_df.iloc[j,0] in list(Forecasts.columns):
					suggested_model = Evaluation_df.iloc[j,0]
					break
			
			# form dataframe for the suggested model
			suggested_df = Forecasts[['Datetime',suggested_model]]
			suggested_df.rename(columns={suggested_model:'Prices €/MWh'}, inplace=True)
			# add suggested model and predictions to dictionary
			SuggPredictions[zone][start]['model'] = suggested_model
			SuggPredictions[zone][start]['preds'] = suggested_df
			# print results
			print('Suggested model as found the best one in the last',daysBack,'evaluation days is:',suggested_model)
			print('Suggested predictions for', (Today + timedelta(days=1)).strftime('%d/%m'),'\n',suggested_df)
			
		except Exception as e:
			print('Exception found in custom models:\n',e)

		# set Datetime as index
		Forecasts = Forecasts.set_index('Datetime')
		# round numerical data after transforming to float64
		Forecasts[Forecasts.select_dtypes(include=np.number).columns] = round(Forecasts[Forecasts.select_dtypes(include=np.number).columns].apply(pd.to_numeric, errors='coerce', axis=1), 2)
		# create a copy of datafram to upload
		UploadForecasts = Forecasts.copy()
		# exclude Naive model if included, from uploading to database as it can be found in Prices tables
		try:
			UploadForecasts.drop(columns=['Naive'], axis=1, inplace=True)
		except:
			pass
			
	#------- Upload forecasts to MySQL
		MySQL_functions.UpdateTable(MySQL_info, TableForecasts, UploadForecasts)
		# Create forecasts table if needed in MySQL
		#MySQL_functions.CreateTable(MySQL_info, TableForecasts, Forecasts, data_type = 'VARCHAR(50)')


print('\n\n---------------------------- end of predictions ----------------------------------')

Current project name: DE_Prices_v1_

Prediction starting timestamp: 2024-02-04 00:00:00  - ending timestamp: 2024-02-04 23:00:00
Query created: SELECT Datetime,Load_DE, Wind_DE, Solar_DE, Hydro_DE, Nuc_DE FROM Fundamentals_forecast WHERE DATE(Datetime) BETWEEN "2024-02-04 00:00:00" AND "2024-02-04 23:00:00";
Query created: SELECT DeliveryStart, SettlementPrice as THE FROM MORE_Trading.GasSpot where Market="THE" AND DeliveryStart <= "2024-02-04 00:00:00" ORDER BY DeliveryStart DESC LIMIT 1;
Query created: SELECT Datetime, EUA_DecLastPrice FROM MORE_Trading.EUA WHERE EUA_DecLastPrice IS NOT NULL AND Datetime <= "2024-02-04 00:00:00" ORDER BY Datetime DESC LIMIT 1;
Query created: SELECT Datetime,DE_LU FROM Prices WHERE DATE(Datetime) BETWEEN "2022-06-09" AND "2024-02-03 00:00:00";

Missing values:
 Datetime            0
Load_DE             0
Wind_DE             0
Solar_DE            0
Hydro_DE            0
Nuc_DE              0
THE                 0
EUA_DecLastPrice    0
dtype: int64 
Tab

# Teams notifications

In [ ]:
#------- Consolidate suggested predictions
# Define start and end date to retrieve data from MySQL database
try:
	start_time = pd.to_datetime(starting_date, format='%d/%m/%Y')
except NameError as ne:
	start_time = datetime.now()
# define today's date
Today = pd.to_datetime(start_time.strftime('%Y-%m-%d'), format='%Y-%m-%d')
# define day ahead date start
start = (Today + timedelta(days=1) ).strftime('%Y-%m-%d %H:%M:%S')

# Initiate data frame to save forecasts
Teams_df = pd.DataFrame(pd.date_range(start, periods=24*days_ahead, freq='H'), columns=['Datetime'])
# Initiate list to store suggested models
suggested_models = []
# loop to consolidate predictions
for zone in zones:
	dates = list(SuggPredictions[zone].keys())
	df = pd.DataFrame(columns=['Datetime', 'Prices €/MWh'])
	for date in dates:
		df = pd.concat([df, SuggPredictions[zone][date]['preds']], axis=0)
 
	df.rename(columns={'Prices €/MWh':zone}, inplace=True)
	df.reset_index(inplace=True, drop=True)
	df['Datetime'] = pd.to_datetime(df["Datetime"], format='%Y-%m-%d %H:%M:%S')
	Teams_df = reduce(lambda x, y: pd.merge(x,y, on='Datetime'),[Teams_df,df])
	suggested_models.append(SuggPredictions[zone][date]['model'])

# reset index
Teams_df.set_index('Datetime', inplace=True)
# add averages at the end of the dataframe
Teams_df.loc[('Average prices'), :] = Teams_df.mean(numeric_only=True).round(2)
print(Teams_df)

#------- Teams format
# Teams URL
URL_Teams = r'https://motoroil.webhook.office.com/webhookb2/75e42a1a-7489-4c96-ba48-271ddabd59aa@b1e32279-3c8b-4c06-9279-43fd8b1c329b/IncomingWebhook/76b43fcb38874355b59a1e452a510762/f45c382a-bdb5-420c-b723-08d7eadb0bdd'

day_ref = 'day'
# form layout text for teams for the suggested models 
if days_ahead>1:
	day_ref = 'days'
if len(zones)>1:
	suggested_models.append('respectively')
Suggested_models = ', '.join(suggested_models)

# define title content
title = str("{}, {} {} ahead price predictions are ready!".format( (Today + timedelta(days=1)).strftime('%d/%m'), days_ahead, day_ref ) )

# define dataframe content
pd.set_option('colheader_justify', 'center')
Teams_df.reset_index(drop=False, inplace=True)
preds_html = Teams_df.to_html(index=False, justify='justify-all').replace('<td>', '<td align="center">')

# define html content
content = f"""
<br>

<ul>
	{preds_html}
</ul>
<br>
<i> Suggested models: {Suggested_models} as evaluated for the last {daysBack} days </i>
<br>

"""

#------- Send to Teams
if Teams_notification:
	# Send content
	send_teams(URL_Teams, content, title)
	print('\n\n> Predictions were posted to Teams!')
